In [28]:
from keras.applications import inception_v3

model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

In [94]:
layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 1.,
    'mixed4': 3.,
    'mixed5': 4,
}

In [95]:
from keras import backend as K

layer_dict = dict([(layer.name, layer) for layer in model.layers])
loss = K.variable(0.)
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss = loss + ((coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :]))) / scaling)

In [96]:
dream = model.input
grads = K.gradients(loss, dream)[0]
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)
def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values
def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss value at', i, ':', loss_value)
        x += step * grad_values
    return x

In [97]:
import scipy
import scipy.misc
from keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)
def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    scipy.misc.imsave(fname, pil_img)
def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [98]:
import numpy as np
import imageio

step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20
max_loss = 10.
base_image_path = './dataset/elephants.jpg'
img = preprocess_image(base_image_path)
original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)
successive_shapes = successive_shapes[::-1]
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])
for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                      iterations=iterations,
                      step=step,
                      max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img
    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    imageio.imwrite('dataset/dream_at_scale_' + str(shape) + '.jpg', img[0])

imageio.imwrite('dataset/final_dream.jpg', img[0])

Processing image shape (160, 241)
...Loss value at 0 : 1.281711
...Loss value at 1 : 1.8371537
...Loss value at 2 : 2.4957883
...Loss value at 3 : 3.0603065
...Loss value at 4 : 3.6772795
...Loss value at 5 : 4.2366323
...Loss value at 6 : 4.6665554
...Loss value at 7 : 5.214514
...Loss value at 8 : 5.638151
...Loss value at 9 : 6.163439
...Loss value at 10 : 6.6280956
...Loss value at 11 : 7.1252804
...Loss value at 12 : 7.3685474
...Loss value at 13 : 8.163757
...Loss value at 14 : 8.528556
...Loss value at 15 : 9.0585575
...Loss value at 16 : 9.366744
Lossy conversion from float32 to uint8. Range [-1.8790994882583618, 1.1801446676254272]. Convert image to uint8 prior to saving to suppress this warning.
Processing image shape (225, 338)
...Loss value at 0 : 2.2962909
...Loss value at 1 : 3.5368357
...Loss value at 2 : 4.526987
...Loss value at 3 : 5.302583
...Loss value at 4 : 6.0850244
...Loss value at 5 : 6.8609896
...Loss value at 6 : 7.430675
...Loss value at 7 : 8.068256
...Loss